In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")

%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0,"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre")
            
import torch
torch.set_printoptions(precision=10)


import yaml
torch.set_float32_matmul_precision('high')
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import Draw
# load model 
from datasets.dataset_utils import  fp_loader_configer

fp_loader_configer.select_version("Hash_Entropy")
fp_loader = fp_loader_configer.fp_loader

import numpy as np 
import random
seed=2
torch.cuda.manual_seed_all(seed) 
np.random.seed(seed)
random.seed(seed)



choosing Hash_Entropy_FP_loader


In [2]:
import pickle
with open(f'/root/gurusmart/MorganFP_prediction/inference_data/coconut_loutus_hyun_training/inference_metadata_latest_RDkit.pkl', 'rb') as file:
    smiles_and_names = pickle.load(file)
print(len(smiles_and_names))

526316


In [3]:
from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.optional_2d_folder_dataset import OptionalInputDataModule



In [4]:



loader_idx_to_name_mapping = {
    0: "eHSQC, H NMR, and C NMR",
    1: "eHSQC and H NMR",
    2: "eHSQC and C NMR",
    3: "eHSQC Only",
    4: "C NMR and H NMR",
    5: "H NMR Only",
    6: "C NMR Only",
    7: "Normal HSQC only",
    8: "standard HSQC, H NMR, and C NMR",
    9: "standard HSQC and H NMR",
    10: "standard HSQC and C NMR",
    
    
}

In [5]:
max_radius = 6

rankingset_path = f"/root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_{max_radius}_dim_16384_stacked_together/FP.pt"

rankingset_data = torch.load(rankingset_path).to("cuda")

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:315: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  result = torch.sparse_compressed_tensor(


In [6]:

def test_extra_input_types(checkpoint_path):
    # checkpoint_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r0_r6_trial_1/checkpoints/epoch=95-step=21696.ckpt"
    checkpoint_path = Path(checkpoint_path)
    model_path = checkpoint_path.parents[1]
    hyperpaerameters_path = model_path / "hparams.yaml"

    with open(hyperpaerameters_path, 'r') as file:
        hparams = yaml.safe_load(file)
        
    del hparams['checkpoint_path'] # prevent double defition of checkpoint_path

    fp_loader = fp_loader_configer.fp_loader
    hparams['num_workers'] = 0
    model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, fp_loader = fp_loader,  **hparams)
    max_radius = int(hparams['FP_choice'].split("_")[-1])
    fp_loader.setup(hparams['out_dim'], max_radius)

    datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], fp_loader = fp_loader_configer.fp_loader, batch_size=1, parser_args=hparams)
    datamodule.setup("test")
    # loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = datamodule.test_dataloader()
    test_dataloaders = datamodule.test_dataloader()

    import tqdm
    model.setup_ranker()
    model.ranker.data = model.ranker.data.to(model.device)
    

    from collections import defaultdict

    ## with MW
    mean_rank_records = defaultdict(lambda: defaultdict(list))

    for i in range(11):
        # if i<7:
        #     # loading from old records
        #     continue
        loader_idx = i
        if i == 7: # standard HSQC only
            loader_idx = 3
        if i == 8: # standard HSQC, H NMR, and C NMR
            loader_idx = 0
        if i == 9: # standard HSQC and H NMR
            loader_idx = 1
        if i == 10: # standard HSQC and C NMR
            loader_idx = 2
            
        for idx, batch in tqdm.tqdm(enumerate(test_dataloaders[loader_idx])):
            batch[0] = batch[0].to(model.device)
            batch[1] = batch[1].to(model.device)
            batch[2] = batch[2].to(model.device)
            
            if i >= 7:
                # last column all zeros
                batch[0][:, :, -1] = 0
            with torch.no_grad():
                result =  model.test_step(batch, 0, dataloader_idx=loader_idx)
            for k,v in result.items():
                
                mean_rank_records[loader_idx_to_name_mapping[i]][k].append(v)
                
                
    for k,v in mean_rank_records.items():
        for k2, v2 in v.items():
            mean_rank_records[k][k2] = np.mean(v2)
            
    return mean_rank_records

In [7]:
three_model_mean_rank_records = []
for path in [
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r0_r6_trial_1/checkpoints/epoch=95-step=21696.ckpt",
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r0_r6_trial_2/checkpoints/epoch=91-step=20792.ckpt",
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r0_r6_trial_3/checkpoints/epoch=67-step=46104.ckpt"
]:
    mean_rank_records = test_extra_input_types(path)
    three_model_mean_rank_records.append(mean_rank_records)

Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
finish entropy list
Hash_Entropy_FP_loader is setup, out_dim=16384, max_radius=6
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_6_dim_16384_stacked_together/FP.pt


4056it [02:05, 32.32it/s]
4056it [02:01, 33.40it/s]
4056it [01:58, 34.35it/s]
4056it [02:03, 32.85it/s]
4056it [02:04, 32.50it/s]
4056it [02:05, 32.43it/s]
4056it [02:00, 33.62it/s]
4056it [01:45, 38.38it/s]
4056it [01:32, 43.83it/s]
4056it [01:34, 43.15it/s]
4056it [01:26, 47.13it/s]
Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_6_dim_16384_stacked_together/FP.pt


4056it [01:28, 46.05it/s]
4056it [01:33, 43.59it/s]
4056it [01:28, 45.79it/s]
4056it [01:28, 46.04it/s]
4056it [01:30, 44.71it/s]
4056it [01:30, 44.80it/s]
4056it [01:29, 45.32it/s]
4056it [01:29, 45.45it/s]
4056it [01:42, 39.65it/s]
4056it [01:30, 45.06it/s]
4056it [01:36, 42.08it/s]
Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Hash_Entropy_FP_loader is already setup
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/non_collision_FP_rankingset_max_radius_6_dim_16384_stacked_together/FP.pt


4056it [01:35, 42.50it/s]
4056it [01:34, 42.81it/s]
4056it [01:30, 45.06it/s]
4056it [01:37, 41.46it/s]
4056it [01:28, 45.72it/s]
4056it [01:32, 43.71it/s]
4056it [01:40, 40.34it/s]
4056it [01:34, 42.79it/s]
4056it [01:33, 43.33it/s]
4056it [01:33, 43.44it/s]
4056it [01:31, 44.23it/s]


In [13]:
from collections import defaultdict
import numpy as np

# Assume your data is stored in three_model_mean_rank_records
# Initialize final result dictionary
average_metrics = defaultdict(dict)

# Get all dataset names (e.g., 'standard HSQC, H NMR, and C NMR', etc.)
dataset_names = three_model_mean_rank_records[0].keys()

for dataset in dataset_names:
    # For each metric under this dataset
    metrics = three_model_mean_rank_records[0][dataset].keys()
    
    for metric in metrics:
        # Collect metric value from all 3 models
        values = [model[dataset][metric] for model in three_model_mean_rank_records]
        average_metrics[dataset][metric] = (float(np.mean(values)), float(np.std(values)))

# Print or use `average_metrics` as needed
for dataset, metrics in average_metrics.items():
    print(f"\n{dataset}")
    for metric_name, avg_val in metrics.items():
        print(f"  {metric_name}: {avg_val[0]:.4f} ± {avg_val[1]:.4f}")



eHSQC, H NMR, and C NMR
  ce_loss: 0.0023 ± 0.0001
  pos_loss: 0.0016 ± 0.0001
  neg_loss: 0.0008 ± 0.0000
  pos_neg_loss: 0.0023 ± 0.0001
  cos: 0.8828 ± 0.0008
  jaccard: 0.8050 ± 0.0012
  active_bits: 51.7434 ± 0.2304
  f1: 0.8812 ± 0.0009
  precision: 0.9036 ± 0.0018
  recall: 0.8654 ± 0.0022
  accuracy: 0.9994 ± 0.0000
  mean_rank: 244.8447 ± 49.2904
  rank_1: 0.5143 ± 0.0095
  rank_5: 0.8263 ± 0.0066
  rank_10: 0.8931 ± 0.0045

eHSQC and H NMR
  ce_loss: 0.0027 ± 0.0001
  pos_loss: 0.0018 ± 0.0001
  neg_loss: 0.0009 ± 0.0000
  pos_neg_loss: 0.0027 ± 0.0001
  cos: 0.8635 ± 0.0011
  jaccard: 0.7777 ± 0.0016
  active_bits: 51.4017 ± 0.1077
  f1: 0.8615 ± 0.0012
  precision: 0.8877 ± 0.0006
  recall: 0.8434 ± 0.0017
  accuracy: 0.9993 ± 0.0000
  mean_rank: 413.8827 ± 60.5148
  rank_1: 0.4594 ± 0.0118
  rank_5: 0.7767 ± 0.0070
  rank_10: 0.8529 ± 0.0056

eHSQC and C NMR
  ce_loss: 0.0024 ± 0.0001
  pos_loss: 0.0016 ± 0.0001
  neg_loss: 0.0008 ± 0.0000
  pos_neg_loss: 0.0024 ± 0.0001


In [9]:
save_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre/inference/flexible_models_avg_by_3"
import pickle
with open(save_path, 'wb') as f:
    pickle.dump(average_metrics, f)

In [10]:
mean_rank_records

defaultdict(<function __main__.test_extra_input_types.<locals>.<lambda>()>,
            {'eHSQC, H NMR, and C NMR': defaultdict(list,
                         {'ce_loss': np.float64(0.0021850935046854713),
                          'pos_loss': np.float64(0.0013887740531379677),
                          'neg_loss': np.float64(0.0007963194544856048),
                          'pos_neg_loss': np.float64(0.00218509350937197),
                          'cos': np.float64(0.8831354480501287),
                          'jaccard': np.float64(0.8051829311052517),
                          'active_bits': np.float64(52.06262327416174),
                          'f1': np.float64(0.8815404801855425),
                          'precision': np.float64(0.9012207794398128),
                          'recall': np.float64(0.8682958531125469),
                          'accuracy': np.float64(0.9993767973469089),
                          'mean_rank': np.float64(247.32051282051282),
                       